# Infraestructure Manager revenue maximization with GSA

## 0. Load libraries

### WARNING!

Install local robin module: `python3 -m pip install -e /Users/david/PycharmProjects/robin`

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from pathlib import Path

from benchmarks.robin_railway import RevenueMaximization
from src.entities import GSA

from random import randint, seed
from robin.kernel.entities import Kernel
from robin.scraping.entities import SupplySaver
from robin.services_generator.entities import ServiceGenerator
from robin.supply.entities import Supply

/Users/uclm/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
seed(21)

pop_size = 20
iters = 50

supply_config_path = Path("../configs/generator/supply_config.yml")
generator_config_path = Path("../configs/generator/generator_config.yml")

supply_save_path = '../configs/gsa_results/'
robin_save_path = '../data/output/robin/'

if not Path(supply_save_path).exists():
        Path(supply_save_path).mkdir(parents=True)

best_solution = {}
max_iters = 10
for i in range(1, max_iters + 1):
    print(f'Iteration {i}')
    
    # Generate random RU's service requests
    generator = ServiceGenerator(supply_config_path=supply_config_path)
    
    generator_seed = randint(1, 1000)
    print(f'\tGenerator seed: {generator_seed}')
    req_file_name = f'supply_gen_{i}'
    services = generator.generate(file_name=Path(f'../data/{req_file_name}.yml'),
                                  path_config=generator_config_path,
                                  n_services=5,
                                  seed=generator_seed)
    
    # Get optimized & feasible timetable
    print(f'../data/{req_file_name}.yml')
    sm = RevenueMaximization(Path(f'../data/{req_file_name}.yml'), safe_headway=10)

    gsa_algo = GSA(objective_function=sm.get_fitness_gsa,
                   is_feasible=sm.feasible_services_times,
                   custom_repair=sm.custom_repair,
                   r_dim=len(sm.boundaries.real),
                   d_dim=0,
                   boundaries=sm.boundaries)
    
    gsa_seed = randint(1, 1000)
    print(f'\tGSA seed: {gsa_seed}')
    #gsa_algo.set_seed(seed=gsa_seed)
    
    training_history = gsa_algo.optimize(population_size=pop_size,
                                         iters=iters,
                                         chaotic_constant=False,
                                         repair_solution=True,
                                         initial_population=sm.get_initial_population(pop_size),
                                         verbose=False)
    
    tt_file_name = f'supply_{i}'
    SupplySaver(services).to_yaml(filename=f'{tt_file_name}.yml', save_path=supply_save_path)
    
    # Simulate market
    path_config_supply = f'{supply_save_path}{tt_file_name}.yml'
    path_config_demand = '../configs/demand/demand.yml'
    path_output_csv = f'{robin_save_path}output_{i}.csv'
    
    if not Path(path_output_csv).parent.exists():
        Path(path_output_csv).parent.mkdir(parents=True)
        
    seed = 21
    kernel = Kernel(path_config_supply, path_config_demand)
    services = kernel.simulate(output_path=path_output_csv, departure_time_hard_restriction=True)
    
    # Evaluation - Better if all RU's income is maximized
    df = pd.read_csv(path_output_csv)
    supply = Supply.from_yaml(path_config_supply)

    services_tsp = {service.id: service.tsp.name for service in supply.services}
    df['tsp'] = df['service'].apply(lambda service_id: services_tsp.get(service_id, np.NaN))
    tsp_revenue = df.groupby('tsp').agg({'price': 'sum'}).to_dict()['price']
    print(tsp_revenue)
    
    improved_solution = False
    if not best_solution:
        best_solution = tsp_revenue
        improved_solution = True
    else:
        if len(tsp_revenue) > len(best_solution):
            best_solution = tsp_revenue
            improved_solution = True
        elif all([tsp_revenue[tsp] > best_solution.get(tsp, -np.inf) for tsp in tsp_revenue]):
            best_solution = tsp_revenue
            improved_solution = True
    
    if improved_solution:
        print("#"*50)
        print(f'\tImproved solution: {best_solution}')
        print(f'\tFile: {tt_file_name}')
        print("#"*50)

Iteration 1
	Generator seed: 169
../data/supply_gen_1.yml
	GSA seed: 506
{'OUIGO': 13334.500000000002, 'Renfe AVE': 38991.68, 'Renfe AVLO': 25189.280000000002}
##################################################
	Improved solution: {'OUIGO': 13334.500000000002, 'Renfe AVE': 38991.68, 'Renfe AVLO': 25189.280000000002}
	File: supply_1
##################################################
Iteration 2
	Generator seed: 858
../data/supply_gen_2.yml
	GSA seed: 31
{'IRYO': 27676.969999999998, 'Renfe AVE': 38710.68, 'Renfe AVLO': 11580.0}
Iteration 3
	Generator seed: 148
../data/supply_gen_3.yml
	GSA seed: 256
{'OUIGO': 13334.500000000002, 'Renfe AVE': 20003.0, 'Renfe AVLO': 41268.15}
Iteration 4
	Generator seed: 506
../data/supply_gen_4.yml
	GSA seed: 509
{'IRYO': 53878.63, 'Renfe AVLO': 12150.86}
Iteration 5
	Generator seed: 574
../data/supply_gen_5.yml
	GSA seed: 699
{'IRYO': 13579.5, 'OUIGO': 27607.940000000002, 'Renfe AVE': 44139.43}
##################################################
	Improved

In [10]:
from robin.demand.entities import Demand

demand = Demand.from_yaml('../configs/demand/demand.yml')

In [11]:
passengers = demand.generate_passengers()

In [12]:
p = passengers[0]

In [19]:
values = [float(rule.split(' ')[-1]) for r, rule in p.user_pattern._rules.items()]
values

[30.0, 40.0, 30.0]

In [44]:
import random

def update_rules(values: list, max_change: int=2):
    for _ in range(len(values)):
        # Set two random indexes
        idx1, idx2 = random.sample(range(len(values)), 2)
        
        # Calculate the amount of change
        cambio = np.round(random.uniform(1, min(max_change, values[idx1], values[idx2])))
        
        # Adjust the values
        values[idx1] -= cambio
        values[idx2] += cambio    
    return values

# Original
original = [30.0, 40.0, 30.0]

updated = update_rules(original.copy())
updated

[29.0, 37.0, 34.0]

100.0
